
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Control your Thymio in Python - Introduction to the tdmclient Library<br/>
    <span style="text-decoration:underline;font-weight:bold;">How to use this notebook?</span><br/>
    This notebook is made of text cells and code cells. The code cells have to be <strong>executed</strong> to see the result of the program. To execute a cell, simply select it and click on the "play" button (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) in the tool bar just above the notebook, or type <code>shift + enter</code>. It is important to execute the code cells in their order of appearance in the notebook.<br/>
</div>

Please note that this notebook is based on the Control your Thymio notebook from Fall 2021 and the Thymio_python_start notebook available here https://www.robot-advance.com/EN/actualite-python-with-thymio-complete-guide-228.htm 
<br/>

# 1. Installations

Tdmclient is a python package that will allow you to connect your Thymio II robot via the Thymio Device Manager (component of the Thymio Suite). This notebook is based on the tdm client tutorial https://pypi.org/project/tdmclient/, where you can find more details about the package. 

To complete this tutorial, you will need to <span style="color:orange"> **install Thymio Suite** (https://www.thymio.org/program/), **Python 3** (https://www.python.org/downloads/), and **pip3 - the package installer for Python** - (https://pypi.org/project/pip/).</span>

To install the tdm client, run the next cell. 

Note : you may need to restart the kernel after installing the new packages

In [6]:
#Install the tdmclient package:
!pip3 install tdmclient --upgrade

## Note : Upgrading the Thymio firmware

Note: *if you want to be able to access and control the LEDs from the serial port, you will have to upgrade the Thymio's firmware to the version 13. This however is not necessary but if you think that you need it then you will have to install Thymio suite, launch Aseba Studio and then right click on the Thymio and select upgrade firmware.*

# 2. Getting started with tdmclient

## 2.1. Pre-requisits

<span style="color:orange">**Open Thymio Suite**</span>


To be able to use the client you first need to open Thymio suite (in order to have a Thymio Device Manager runnning). Only by doing that wil you be able to connect to the robot in python. 

Your **robot should be connected via USB or via the RF dongle**. It is important to note that **only one client can control the robot at the same time**: so we will need to choose if we want to use **Aseba Studio (in the Thymio Suite) or Python (via the tdmclient)**.

If you have already started to control your robot via Aseba Studio, you can unlock the robot by clicking the little lock icon in the tab title near the top left corner of the Aseba Studio window.

## 2.2. Displaying the list of Thymio robots connected to the computer with tdmclient list

We can use the tdmclient to display the list of Thymio robots connected to the computer via the tdmclient list

In [7]:
#The exclamation point (!) allow to execute a terminal command in the notebook:
!python3 -m tdmclient list

id:         ee38ffef-3c84-4f82-84cf-a9cb52844d18
group id:   61522e0f-8393-4dd9-a344-c435682beeac
product id: 8 (Thymio II)
name:       thymio 1
status:     2 (available)
cap:        7
firmware:   14



If you **get any error because python is not found**, please try to replace `python3` by `python` (just remove the 3 in the command line and the following ones).

## 2.3. Observing variable changes

If you want to observe the status of the variables on the Thymio (e.g. for debugging), there are two options

### 2.3.1. In the notebook using tdmclient watch

Display all node changes (variables, events and program in the scratchpad) until control-C is typed. You should type this command line in your command prompt (not supported on the notebook)

## 2.3.2. In a gui using tdm.client gui
Run the variable browser in a window. The GUI is implemented with TK which we install with the following command

In [8]:
!python -m pip install tk-tools

In [9]:
!python3 -m tdmclient gui

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/seby/miniconda3/envs/mondada/lib/python3.12/site-packages/tdmclient/__main__.py", line 67, in <module>
    sys.exit(getattr(tdmclient.tools, tool_name).main(values))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/seby/miniconda3/envs/mondada/lib/python3.12/site-packages/tdmclient/tools/gui.py", line 610, in main
    win.mainloop()
  File "/home/seby/miniconda3/envs/mondada/lib/python3.12/tkinter/__init__.py", line 1505, in mainloop
    self.tk.mainloop(n)
  File "/home/seby/miniconda3/envs/mondada/lib/python3.12/tkinter/__init__.py", line 1963, in __call__
    def __call__(self, *args):

KeyboardInterrupt


At launch, the robot is unlocked, i.e. the variables are just fetched and displayed: Observe is displayed in the status area at the bottom of the window. To be able to change them, activate menu Robot>Control. Then you can click any variable, change its value and type Return to confirm or Esc to cancel.

# 3. Programing the Thymio directly from Jupyter Notebooks

With the following code block, you will activate the synchronisation between the internal variables of Thymio and the notebook's variables. The names are similar, with the '.' replaced by '_' (because dots have a different meaning in Python). For example, the center front sensor variable 'prox.horizontal[2]' becomes 'prox_horizontal[2]' and the variable to set the speed of the left motor 'motor.left.target' becomes 'motor_left_target'. 

In [46]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
await tdmclient.notebook.start()

If you get a `Node lock error`, this means that one client had already locked the robot:
 - first check that the robot **is not already locked in Thymio Suite** (little lock icon in the tab title near the top left corner of the Aseba Studio window).
 - if you cannot unlock the robot in Thymio Suite, you probably need to restart the kernel of the notebook 
 
If there is any code remaining on the Thymio from your previous tries, you can simply turn off/on the robot.

## 3.1 Setting actuator variables

Then the variables which match the robot's are synchronized in both directions, you can simply set their value in python and these will be actualised on the robot

Note that in python, dots *.* are replaced by underscores *_*: "motor.left.target" in Aseba become "motor_left_target" in python.

### Motors
The motors are regulated in speed. There are 3 variables that are used per motor:
- `motors_left_speed` and `motor_right_speed` : read-only. The current speed value that is measured
- `motors_left_pwm` and `motor_right_pwm` : read-only. The pwm value for the control
- `motors_left_target` and `motor_right_target` : the target value set by the user, should range from -500 to 500

The motors variables are updated in Thymio on the event `motor`, 100 times per second.

In [3]:
%%run_python
# turn on the spot
motor_left_target= 200
motor_right_target= 200

In [4]:
# forward
motor_left_target= 200
motor_right_target= 200

In [5]:
# stop
motor_left_target= 0
motor_right_target= 0

In [10]:
motor_left_target = 100

In [11]:
motor_left_target = 0

If you would like to have a function which sets both motor speeds you can create a function to do so like shown below. Beware : you will need to 
- add the decorator **@tdmclient.notebook.sync_var** just before your function to make sure that you variables (see https://github.com/epfl-mobots/tdm-python/blob/main/doc/notebooks.md)
- declare the motor target variables as global variables within the function

An example is shown below

In [45]:
import time
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed
    

motors(100, 100, verbose=True) #test with lower speed value
time.sleep(5)
motors(0, 0, verbose=True)

		 Setting speed :  100 100
		 Setting speed :  0 0


### LEDs

Thymio has many LEDs, which can be accessed either by variables or by native functions. Arguments range from 0 (off) to 32 (fully on). In addition, some constants have been defined for colours in tdmclient: `BLACK`, `BLUE`, `RED`, `GREEN`, `MAGENTA`, `YELLOW`, `CYAN` and `WHITE`.

In [19]:
var=leds_top
print(var)
leds_top = [32,0,32]
print(leds_top)

[0, 0, 32]
[32, 0, 32]


In [18]:
leds_top=[0,0,32] #the two top RGB LEDs as a variable. Arguments are in the order red, green, blue

In [20]:
%%run_python
nf_leds_top(0,0,32) #the two top RGB LEDs accessed by a funtion. Arguments are in the order red, green, blue

In [21]:
leds_bottom_left=[0,32,32] #the bottom left RGB LED as a variable. Arguments are in the order red, green, blue

In [22]:
%%run_python
nf_leds_bottom_left(0,32,32) #the bottom left RGB LED accessed by a funtion.. Arguments are in the order red, green, blue

In [24]:
leds_bottom_right=[0,32,0] #the bottom right RGB LED as a variable. Arguments are in the order red, green, blue

In [25]:
%%run_python
nf_leds_bottom_right(0,32,0) #the bottom right RGB LED accessed by a funtion.. Arguments are in the order red, green, blue

In [ ]:
leds_circle=[0,4,8,12,16,20,24,28] #the circle of LEDs around the buttons as a variable. The first arguments is the front one, then clockwise.

In [ ]:
%%run_python
nf_leds_circle(0,4,8,12,16,20,24,28) #the circle of LEDs around the buttons accessed by a funtion.. The first arguments is the front one, then clockwise.

In [ ]:
%%run_python
nf_leds_prox_h(32,24,16,8,0,8,16,32) # the LEDs next to horizontal proximity sensors. The first argument is the front left LED, then clockwise.

In [ ]:
%%run_python
nf_leds_prox_v(32,8) # the LEDs next to ground sensors. The first argument is the front left LED, then clockwise.

In [ ]:
%%run_python
nf_leds_buttons(32,24,16,8) # the 4 LEDs next to the arrow buttons. The first argument is the left LED, second is the right LED.

In [ ]:
%%run_python
nf_leds_rc(32) # the LED next to RC receiver.

In [ ]:
%%run_python
nf_leds_temperature(32, 16) # the LEDs next to temperature sensor. First red, second blue.

In [44]:
%%run_python
nf_leds_rc(32) # the LED next to microphone.

KeyError: 'error_msg'

## 4.2. Getting sensor values

The thymio has multiple sensors: 
- buttons
- proximity sensors
- accelerometer
- microphone
- temperature sensor
- remote control receiver

We present a few here but feel free to look up the others if these would interest you for the project later in the semester

### Buttons
The state of the buttons can be read through variables. The value is either 1 (touched) or 0 (not touched). Try touching the buttons while executing the code below.
The buttons states are updated in Thymio on the event `buttons`, 50 times per second. In addition, the events `button_center`, `button_forward`, `button_backward`, `button_left` and `button_right` are triggered when the respective buttons are touched, or released.

In [47]:
print('Center button: ',button_center)
print('Forward button: ',button_forward)
print('Backward button: ',button_backward)
print('Left button: ',button_left)
print('Right button: ',button_right)

Center button:  0
Forward button:  0
Backward button:  0
Left button:  0
Right button:  1


### Proximity sensors
The state of the proximity sensors can be read through variables. The treatment is a bit different for the horizonal and vertical proximity sensors. The sensor values are updated in Thymio on the event `prox`, 10 times per second. Proximity sensors work with infrared light: they emit a beam of IR light and measure the light that is reflected. An object placed close will reflect more light than one placed far away.

#### Horizontal proximity sensors
The value is either 0 (nothing is in the field of view) or ranges between 1000 (far away) and 4000+ (very close). The variable `prox_horizontal` is a table of 7 entries for the 7 horizontal sensors, starting with the front left sensor and then in clockwise order. Try placing your hand or an object in front of or behind the robot while executing the code below, to see the values change.

In [52]:
print('Proximity sensors values: ', prox_horizontal)

Proximity sensors values:  [1618, 2282, 1618, 0, 0, 0, 0]


#### Vertical proximity sensors
The vertical proximity sensors (often called ground sensors) are treated a bit differently. 
<br>They have 3 variables associated: `prox_ground_ambiant`, which measures the ambiant light, `prox_ground_reflected`, which measures the light when the sensor is emitting light, and `prox_ground_delta`, the difference of the two (= the light reflected by the object in front). Each variable is a 2-value table, the first one being for the left sensor and the second for the right sensor.
<br>The values range from 0 (nothing detected / no light) to 1023 (something close / lots of light).

In [55]:
print('Ground sensors ambiant light: ', prox_ground_ambiant)

Ground sensors ambiant light:  [0, 0]


In [56]:
print('Ground sensors measure during light emission: ', prox_ground_reflected)

Ground sensors measure during light emission:  [572, 542]


In [57]:
print('Ground sensors delta (light actually reflected by an object in front): ', prox_ground_delta)

Ground sensors delta (light actually reflected by an object in front):  [953, 926]


If you want to create a function which prints the sensor values on a loop you have to use the function decorator **@tdmclient.notebook.sync_var** as for the motors. Here is an example below 

In [ ]:
@tdmclient.notebook.sync_var 
def ground_sensor():
    global prox_ground_reflected
    return prox_ground_reflected

@tdmclient.notebook.sync_var 
def horiz_sensor():
    global prox_horizontal
    return prox_horizontal

@tdmclient.notebook.sync_var 
def print_sensor_values(sensor_func, print_duration=3, delta_time=0.5):
    """
    While the end time has not been reached, print the sensor values every delta_time seconds
    """
    
    t_end = time.time()+print_duration

    while time.time() < t_end:
        time.sleep(delta_time)
        print(sensor_func())

In [ ]:
print_sensor_values(ground_sensor)

[220, 143]
[97, 132]
[6, 31]
[1, 11]
[17, 23]
[15, 25]


In [ ]:
print_sensor_values(horiz_sensor)

[3522, 0, 3627, 4653, 4458, 0, 0]
[3806, 2863, 4022, 4633, 4458, 0, 0]
[3850, 2317, 3514, 4560, 4465, 0, 0]
[3887, 2175, 3328, 4501, 4468, 0, 0]
[3867, 1790, 3068, 4405, 4470, 0, 0]
[3858, 1646, 2951, 4338, 4467, 0, 0]


### Accelerometer
Thymio has a 3-axis accelerometer, which allows to detect its orientation in space, freefall and schocks.
<br>The sensor values are stored in variable `acc`, a 3-value table (left-right axis, front-back axis, up-down axis). It is refreshed on event `acc`, at 16 Hz. The values range from -32 to 32, with 23 corresponding to 1 G. 
<br>In addition, when shocks are detected, event `tap`is triggered.

In [58]:
%%run_python
@onevent
def acc():
    global acc, leds_top
    if acc[0]>18 or acc[0]<-18: #Thymio is blue when placed on one of its sides
        leds_top=[0,0,32]
    if acc[1]>18 or acc[1]<-18: #Thymio is red when placed on its front or backside
        leds_top=[32,0,0]
    if acc[2]>18 or acc[2]<-18: #Thymio is green when placed on its wheels or upside-down
        leds_top=[0,32,0]

In [59]:
%%run_python
@onevent
def tap():
    nf_sound_system(1) #Thymio plays the shutdown sound when hit

# 4. Executing code directly on the Thymio (not executed on your computer)

So far, you only interacted with variables but it is also possible to send a whole program to Thymio, written either in Aseba Python. Previously, the code was executed in python on your computer. Now, we are going to code in python, and then this code will be *transpiled* into Aseba, and then loaded directly on the robot.
> Aseba is the native language used to program the Thymio robot (you can find more about it here http://wiki.thymio.org/en:asebalanguage)

Below, we'll give equivalent examples in both languages: change the robot's led circle activation. 

%%run_python  allows to send the content of the cell (in simple python code) to the robot and run it.
%%run_aseba is the equivalent that allows to send the content of the cell (in aseba code) to the robot and run it.



## 4.1. Simple examples

You can use python code that will be transpiled in Aseba with `%%run_python`:

In [ ]:
%%run_python
v = [32, 0, 32, 0, 32, 0, 32, 0]
leds_circle = v

You can use Aseba code directly with `%%run_aseba`:

In [ ]:
%%run_aseba
var v[] = [32, 32, 32, 0, 0, 0, 32, 32]
leds.circle = v

Finally, `%%transpile_to_aseba` allows you to see how the python code you wrote is transpiled in Aseba:

In [ ]:
%%transpile_to_aseba
v = [30, 0, 32, 0, 32, 0, 32, 0]
leds_circle = v

var v[8]

v = [30, 0, 32, 0, 32, 0, 32, 0]
leds.circle = v



## 4.2. Thymio Events 

> Aseba is an event-based architecture, which means that events trigger code execution asynchronously.
Events can be external, for instance a user-defined event coming from another Aseba node, or internal, for instance emitted by a sensor that provides updated data. from http://wiki.thymio.org/en:asebalanguage#toc15

Here is an example of ASEBA code that will be sent to the robot in order to change the robot's color depending on the button pressed

In [ ]:
%%run_aseba
onevent buttons
    if button.center==1 then
        leds.top=[32,0,0]
    elseif button.forward==1 then
        leds.top=[0,32,0]
    elseif button.backward==1 then
        leds.top=[0,0,32]
    elseif button.left==1 then
        leds.top=[32,32,0]
    elseif button.right==1 then
        leds.top=[0,32,32]
    end

To do the python equivalent, you have to create a function for the event and decorate it with `@onevent`. This will result in the execution of the function every time the event is triggered. 
If Thymio variables are used inside the event, they should be declared as global (by default they would be considered local variables).



In [ ]:
%%run_python
@onevent
def buttons():
    global button_center, button_forward, button_backward, button_left, button_right, leds_top
    if button_center==1: 
        leds_top=[32,0,0]
    elif button_forward==1: 
        leds_top=[0,32,0]
    elif button_backward==1: 
        leds_top=[0,0,32]
    elif button_left==1: 
        leds_top=[32,32,0]
    elif button_right==1: 
        leds_top=[0,32,32]

Here is another example using the proximity sensor where the objecive is to change the robot's speed depending on the proximity sensor value. 
The event **prox** is generated after every update by the Thymio of the proximity sensors, with a frequency of 10 Hz. 
Also, please note that the variables are all global in Aseba. That means you should not forget to declare global variables at the beginning of each function. If you don't, your python code transpiled could modify a local variable in a function, and not the desired variable `motor_left_target` for example.

In [ ]:
%%run_python

@onevent
def prox():
    global prox_horizontal, motor_left_target, motor_right_target
    prox_front = prox_horizontal[2]
    speed = -prox_front // 10
    motor_left_target = speed
    motor_right_target = speed

Thymio has an array of local events, most of which were already explained. In addition, there are two configurable timers, `timer0` and `timer1`. Their periods are set through the `timer_period` variable in milliseconds.

In [ ]:
%%run_python
timer_period[0]=100 #timer0 will fire 10 times per second
timer_period[1]=1000 #timer1 will fire once per second
toggle0=0
toggle1=0

@onevent
def timer0(): #on timer0 we toggle the color of the top LEDs
    global leds_top, toggle0
    if toggle0:
        toggle0=0
        leds_top=RED
    else:
        toggle0=1
        leds_top=BLACK
    
@onevent
def timer1(): #on timer1 we toggle the color of the bottom LEDs
    global leds_bottom_left, leds_bottom_right, toggle1
    if toggle1:
        toggle1=0
        leds_bottom_left=GREEN
        leds_bottom_right=GREEN
    else:
        toggle1=1
        leds_bottom_left=BLACK
        leds_bottom_right=BLACK  

Here is a table with all the events that you can program

| event | description | frequency (Hz) | result |
| :- | :- | :- | :- |
| button_backward | back arrow was pressed or released | upon action | button.backward |
| button_left | left arrow was pressed or released | upon action | button.left |
| button_center | central button was pressed or released | upon action | button.center |
|button_forward|front arrow was pressed or released|upon action|button.forward|
|button_right|right arrow was pressed or released|upon action|button.right|
|buttons|button values have been probed|50|buttons.backward, buttons.left, buttons.center, buttons.forward, buttons.right|
|prox|proximity sensors were read|10|prox.horizontal[0-7], prox.ground.ambiant[0-1], prox.ground.reflected[0-1] and prox.ground.delta[0-1] |
|prox_comm|value received from IR sensors|upon value reception|prox.comm.rx|
|tap|a shock was detected|upon shock|acc[0-2] |
|acc|the accelerometer was read|16|acc[0-2] |
|mic|ambient sound intensity was above threshold|when condition is true|mic.intensity|
|sound_finished|a sound started by aseba has finished playing by itself|when sound finishes|
|temperature|temperature was read|1|temperature|
|rc5|the infrared remote-control receiver got a signal|upon signal reception|rc5.address and rc5.command|
|motor|PID is executed 	100|motor.left/right.speed, motor.left/right.pwm |
|timer0|when timer 0 period expires|user-defined||
|timer1|when timer 1 period expires|user-defined||

### Global events

Users can also configure global events, which are not received locally but are set to other nodes in the network in Aseba. In our case, this allows to send event data to the computer. In Python for Thymio, global events do not need to be declared. They are sent with the `emit("name")` or `emit("name", param1, param2, ...)` to emit an event without or with parameters. The first argument must be a literal string, delimited with single or double quotes. The remaining arguments, if any, must be scalar expressions and are passed as event data.

Let's see the following example. Let's imagine that we want to send a measure of the temperature each time the front button is pressed:

In [ ]:
%%run_python --wait
#this is the code that will run on the robot

@onevent
def button_forward(): #every time we touch the forward button, a measure of the temperature is made and sent to the computer
    if button_forward==1: 
        global temperature
        emit("Temp", temperature)
    
    
@onevent
def button_center(): #center button to exit
        exit()


On the computer, the event data can be retrieved with the function `get_event_data(event_name)`. To clear the past events, use `clear_event_data()`.

After executing the code above, press the forward button on Thymio y few times to collect measures, then the center button to exit. Then execute the line below to retrieve the collected data.

In [ ]:
get_event_data("Temp")

In [ ]:
clear_event_data()

Custom event can also be sent from the computer to the robot. Let's see an example below where the robot has its own behaviour but reacts to the event sent from the computer:

In [ ]:
%%run_python

timer_period[0]=1000 #timer0 will fire once per second
toggle=0

#behaviour of the robot, turning and blinking
@onevent
def timer0(): #on timer0 we toggle the color of the top LEDs
    global leds_top, toggle, motor_left_target, motor_right_target
    if toggle:
        toggle=0
        motor_left_target=100
        motor_right_target=-100
        leds_top=RED
    else:
        toggle=1
        motor_left_target=-100
        motor_right_target=100
        leds_top=BLUE
        
#custom event, sent from the computer: stop until next time the timer fires
@onevent
def shortbreak():
    global leds_top, motor_left_target, motor_right_target
    motor_left_target=0
    motor_right_target=0
    leds_top=GREEN
    


Once this is running on the robot, execute the following cell a few times to see how it turns Thymio green and stops it until the next timer event.

In [ ]:
send_event("shortbreak")

Use the next cell to stop the robot.

In [ ]:
%%run_python
motor_left_target=0
motor_right_target=0

### Exit and print: special cases of global event
In tdmclient, two functions are available, that do not exist in the native functions of Thymio. `print("text", exp, ...)` allows to print to the notebook a string followed by one or more values. 
Instead of just `%%run_python`, the program cell must begin with `%%run_python --wait` in order to run as long as required to process events. If you do not want to wait until the end of the execution you can force stop by clicking on the Stop button of Jupyter (interrupt the kernel).
`exit()` then allows to end the execution of a cell (useful when we used `--wait` at the beginning of the cell). 

These two functions actually send global events from the robot to the notebook, that are then recognized and used to print something or exit the cell.

In [ ]:
%%run_python --wait

@onevent
def button_forward(): #when we touch the forward button, we print something based on the internal state of the robot
    if button_forward==1:
        print("Something based on the internal state of the robot", acc[2]*prox_ground_delta[0], button_forward)
        
@onevent
def button_center(): #once we're done sending events, the center button will let us exit the cell
    if button_center==1:
        exit()
    

Here is another example which makes use of the Thymio's timer and alternates the color of the robot between CYAN and off according to the value of i while sending a print statement to the notebook

In [ ]:
%%run_python --wait

i = 0

timer_period[0] = 1000

@onevent
def timer0():
    global i, leds_top
    i += 1
    is_odd = i % 2 == 1
    if is_odd:
        print(i, "odd")
        leds_top = [0, 32, 32]
    else:
        print(i, "even")
        leds_top = [0, 0, 0]
        
    if i> 5:
        exit()

## Advanced concept: custom events

You can send events from the notebook to the robot. This can be useful for instance if you implement a low-level behavior on the robot, such as obstacle avoidance and sensor acquisition, and send at a lower rate high-level commands which require more computing power available only on the PC.

The Thymio program below listens for events named `color` and changes the top RGB led color based on a single number. Bits 0, 1 and 2 represents the red, green, and blue components respectively.

In [ ]:
%%run_python

@onevent
def color(c):
    global leds_top
    leds_top[0] = 32 if c & 1 else 0
    leds_top[1] = 32 if c & 2 else 0
    leds_top[2] = 32 if c & 4 else 0

Now that the program runs on the robot, we can send it `color` events. The number of values in `send_event` should match the `@onevent` declaration. They can be passed as numeric arguments or as arrays.

In [ ]:
for col in range(8):
    send_event("color", col)
    sleep(0.5)

It is also possible to *collect data from the robot using custom events*. You can have a look here https://pypi.org/project/tdmclient/#description (search custom events, at the very end of the tutorial).

Finally, you can stop the notebook environment, in order to unlock the robot. This will be very useful as we are going to use another method in the next part of the tutorial.

In [62]:
await tdmclient.notebook.stop()

# 5. Executing an existing ASEBA program on the Thymio

## 5.1. Using an ASEBA file with tdmclient run

Execute an Aseba program on the Thymio:

In this example, we are going to program the blink.aseba program on the thymio using the following command

In [66]:
!python3 -m tdmclient run --scratchpad src/blink.aseba

The option `--scratchpad` allow the code to be also displayed in the Aseba Studio (where you are able to observe some fetaures of the robot). 

And then you can stop it:

In [64]:
!python3 -m tdmclient run --stop

## 5.2. Using a python file with tdmclient run

To avoid having to learn the Aseba language, it is now possible to program the robot with Python files. Note however that only a subset of Python functionalities can be used:

*Note: it seems that this command will run forever, so if you want to test it, you should press the notebook button stop the kernel to stop the cell and be able to run the following ones. In the command lines just press Ctrl+C. It should be fixed soon in the next tdmclient version.*

Don't forget to stop it:

# 6. Asynchronous programming to control the Thymio from Python

## What is asynchronous programming?

We can also call an **asynchronous** function in such a way that **its result is waited for**. This can be done in a *coroutine*, a special function which is executed at the same time as other tasks your program must perform, with the `await` Python keyword; or handled by the helper function `aw`. Keyword await is valid only in a function (or program), hence we cannot call it directly from the Python prompt.

>An asynchronous program behaves differently. It still takes one execution step at a time. The difference is that the system may not wait for an execution step to be completed before moving on to the next one. This means that the program will move on to future execution steps even though a previous step hasn’t yet finished and is still running elsewhere. This also means that the program knows what to do when a previous step does finish running. from https://realpython.com/python-async-features/#understanding-asynchronous-programming

>In the asynchronous world, things change around a bit. Everything runs on a central event loop, which is a bit of core code that lets you run several coroutines at once. Coroutines run synchronously until they hit an await and then they pause, give up control to the event loop, and something else can happen. from https://www.aeracode.org/2018/02/19/python-async-simplified/

First, we are going to use Python, executed on our computer, to give instructions to the Thymio.
One particular useful class is called `ClientAsync`, and we will understand some of its features:

In [67]:
from tdmclient import ClientAsync, aw

To **connect your Thymio** create a Client Object

In [68]:
#Create a Client Object:
client = ClientAsync()

To check the connexion the client will connect to the TDM (Thymio Device Manager) which will send messages to us, such as one to announce the existence of a robot:

In [69]:
client.process_waiting_messages()

True

The value of `node` is an object which contains some properties related to the robot and let you communicate with it. We can called the node explicitely, its _id_ is displayed when you just print the node:

In [70]:
node = client.nodes[0]
node

Node ee38ffef-3c84-4f82-84cf-a9cb52844d18

## 6.1. Load an Aseba program on the Thymio

A very useful method will allow you to `lock` the robot, so you will be able to change its variables and run programs. Beware that the robot **should no be already locked in Thymio Suite.**

In [71]:
aw(node.lock())

Node ee38ffef-3c84-4f82-84cf-a9cb52844d18

We will then use a little Aseba program, to make the top led of the Thymio blink:

In [72]:
# Define the program
program = """
var on = 0  # 0=off, 1=on
timer.period[0] = 500

onevent timer0
    on = 1 - on  # "on = not on" with a syntax Aseba accepts
    leds.top = [32 * on, 32 * on, 0]
"""

# Compile the program
r = aw(node.compile(program))

print("The compilation is successful :", r==None)

The compilation is successful : True


*The variable `r` will store the result of the call: `None` if the operation was sucessful, an error number otherwise.*

In [73]:
# Run the program that is already compiled:
aw(node.run())

In [74]:
# Stop the program
aw(node.stop())

We can write some text in the Aseba Studio (we can use it to only *observe* the behavior of the robot):

In [ ]:
aw(node.set_scratchpad("Hello, Studio!"))

## 6.2. Passing values to the Thymio

Change the speed:

In [75]:
v = {
    "motor.left.target": [50],
    "motor.right.target": [50],
}
aw(node.set_variables(v))

In [76]:
v = {
    "motor.left.target": [0],
    "motor.right.target": [0],
}
aw(node.set_variables(v))

In [ ]:
#Now unlock the robot:
aw(node.unlock())

## 6.3. Reading values from Thymio sensors

Here you can see how to read the values of the horizontal proximity sensors 10 times with a frequency of 5Hz (once every 0.2s):

In [77]:
await node.wait_for_variables({"prox.horizontal"})
for i in range(10):
    print(list(node.v.prox.horizontal))
    await client.sleep(0.2)

[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]


As you can observe, we are able to obtain updated values here, as it was not possible in option 1.

## Connection summary using tdmclient

Next time you need to use tdmclient ClientAsync and its methods, the simpler way is to use the following code, at the begining of your notebook:

## 6.5. How to create events


Now we are going to see how to create an event:

In [ ]:
aw(node.lock())
# Create an event named speed, with a size of data of 2 (should be between 0-32)
aw(node.register_events([("speed", 2)]))

In [ ]:
# The event data are obtained from variable event.args:
program = """
onevent speed
    motor.left.target = event.args[0]
    motor.right.target = event.args[1]
"""
aw(node.compile(program))
aw(node.run())

`node.send_events` has one argument, a dict where keys correspond to event names and values to event data.

In [ ]:
# turn right
aw(node.send_events({"speed": [40, 20]}))
# wait 1 second
aw(client.sleep(1))
# stop the robot
aw(node.send_events({"speed": [0, 0]}))

In [ ]:
aw(node.unlock())

## 6.4. More about tdmclient and ClientAsync

This section will help you to understand better other functionalities that **would not be required for the exercise sessions, but could be useful for the project for example.**

### List of accessible methods

The object `node` has multiple properties that we can access:

In [ ]:
node.props

We can also use the command `dir()` to get the list of the instances (object attributes) of `node`. We will learn how to use some of the methods which are contained in the list:

In [ ]:
dir(node)

We are going to first have a look at `var`:

In [ ]:
node.var

It is empty right now, we should use first the method `wait_for_variables()`:

In [ ]:
# Get the documentation of a method using __doc__
node.wait_for_variables.__doc__

In [ ]:
aw(node.wait_for_variables())
node.var

Buy using `node.var`, we can understand what the different read-write variables that you can access are. You need to know the name and size of the variables that you are interested in.

# 7. Performance comparison

We are going to execute a simple code, showing the value of the proximity sensors on the leds.circle. To compare the performance of: coding in Aseba, in Python that will be transpiled, or in Python on your own machine, we will use `%%timeit`, that will execute the code several times and show the time of execution.

In [80]:
# Import tdmclient Notebooks environment:
import tdmclient.notebook
await tdmclient.notebook.start()

NodeLockError: Node lock error (current status: busy)

## Aseba

In [ ]:
%%timeit
%%run_aseba

var a
var _tmp[1]
var i

a = 0
while a < 10 do
    i = 0
    while i < 7 do 
        leds.circle[i] = prox.horizontal[i]
        i++
    end
    a++
end

## Python transpiled

In [78]:
%%timeit
%%run_python

for a in range(10):
    for i in range(7):
        leds_circle[i] = prox_horizontal[i]

AttributeError: 'NoneType' object has no attribute 'run_program'

In [ ]:
await tdmclient.notebook.stop()

## Python run on computer

In [81]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

NodeLockError: Node lock error (current status: busy)

In [ ]:
leds = [0, 0, 0, 0, 0, 0, 0, 0]
await node.wait_for_variables({"prox.horizontal"})

In [ ]:
%%timeit

for i in range(10):
    prox = list(node["prox.horizontal"]) + [0]
    leds = prox
    v = {"leds.circle": leds,
        }
    aw(node.set_variables(v))

In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [ ]:
leds = [0, 0, 0, 0, 0, 0, 0, 0]
await node.wait_for_variables({"prox.horizontal"})

In [ ]:
%%timeit

for i in range(10):
    prox = list(node["prox.horizontal"]) + [0]
    leds = prox
    v = {"leds.circle": leds,
        }
    aw(node.set_variables(v))